Modifying annotations in BisQue
======================

We'll be using BisQue APIs and adding and removing annotations

Make sure you have bisque api installed:
> pip install bisque-api

In [ ]:
import os
import csv
from datetime import datetime
try:
    from lxml import etree
except ImportError:
    import xml.etree.ElementTree as etree

Include BisQue API

In [ ]:
from bqapi import BQSession, BQCommError
from bqapi.util import save_blob, localpath2url

define some paths and urls

In [ ]:
image_url = 'http://bisque.ece.ucsb.edu/data_service/00-uLFngDV5SFwHoitPniSqqN'

Initiaize the authenticated session
--------------

Initialize a BisQue session using simple user credentials

In [ ]:
root = 'http://bisque.ece.ucsb.edu'
user = 'demo'
pswd = 'demo'

session = BQSession().init_local(user, pswd,  bisque_root=root, create_mex=False)

Add annotations
------------------------------

We'll add a small tree of textual and numerical annotaions

In [ ]:
t = etree.Element ('tag', name='my annotations')
etree.SubElement (t, 'tag', name='species', value='Arabidopsis Thaliana')
etree.SubElement (t, 'tag', name='confidence', value='0.8', type='number')

print etree.tostring(t, pretty_print=True)

Store annotations to the image
------------------------------------------

In [ ]:
t = session.postxml(image_url, t)

if t is None or t.get('uri') is None:
    print 'Adding annotations failed'
else:
    print 'Tags URL: %s'%t.get('uri')

Any annotation in the tree has a URL and can be operated on with standard HTTP verbs:

* GET - read
* POST - add/modify
* PUT - replace
* DELETE - remove

Fetch parts of the image annotation document
--------------------------------------------

Any of the nodes in the tree can be fetched with various depths:

* view=short - only the node itself
* view=full - the node and the first level of children
* view=deep - the node and all of its children

In [ ]:
image = session.fetchxml(image_url, view='full')
print etree.tostring(image, pretty_print=True)

Modify existing annotations
----------------------------

First find the annotation to modify

In [ ]:
t = image.xpath('//tag[@name="my annotations"]')[0]
print etree.tostring(t, pretty_print=True)

let's modify its name

In [ ]:
t.set('name', 'new annotations')
print etree.tostring(t, pretty_print=True)

Now store the modified element back to the system

In [ ]:
t = session.postxml(t.get('uri'), t)
print etree.tostring(t, pretty_print=True)

Delete an existing annotation
--------------------------------

Let's find the annotation to delete

In [ ]:
g = image.xpath('//gobject[@type="My nuclei"]')[0]
print etree.tostring(g, pretty_print=True)

send DELETE HTTP command to the annotation's URI

In [ ]:
g = session.deletexml(g.get('uri'))
print g